<a id="sectionx"></a>
# Consumer Finance - Demo - Helper Calls
* [Create New Customer](#section0)
* [Add New Card Transaction](#section1)
* [Get Accounts](#section3)
* [Create ALL in 1 go](#section2)



In [19]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

<a id="section0"></a>
[back](#sectionx)
## Create New Customer
* Create a New Customer
* Add a Credit Arrangement to the Customer

In [263]:
%%writefile NEWCUSTTEMP
{
"firstName": "{{FirstName}}",
"lastName": "{{LastName}}",
"preferredLanguage": "ENGLISH",
"addresses": [
    {
      "country": "UK",
      "city": "Liverpool"
    }
],
"notes": "Some Notes on this person",
"gender": "MALE",
  
"assignedBranchKey": "{{BranchID}}"

}

Overwriting NEWCUSTTEMP


In [262]:
def createNewCustomer(fname,lname):
    global newCustomerID, custShortID
    PLACEHOLDERS = {
        "BranchID": "8a818f156ccf5fb1016cd2e8e4532b09",
        "FirstName": fname,
        "LastName": lname
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/clients', headers=HEADERS, params=PARAMS, body="NEWCUSTTEMP", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)

    # Remember the ID of the customer created we will use it below 
    newCustomerID = r.json()['encodedKey']
    custShortID = r.json()['id']
   

In [232]:
createNewCustomer()

API status:
201
Now the JSON:
{"encodedKey":"8a818f746cd2fea7016cd3ace4c74d28","id":"256422253","state":"INACTIVE","creationDate":"2019-08-27T18:00:05+02:00","lastModifiedDate":"2019-08-27T18:00:05+02:00","approvedDate":"2019-08-27T18:00:05+02:00","firstName":"John","lastName":"Smith4","preferredLanguage":"ENGLISH","gender":"MALE","notes":"Some Notes on this person","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","clientRoleKey":"8a818e74677a2e9201677ec2b4c336aa","loanCycle":0,"groupLoanCycle":0,"addresses":[{"encodedKey":"8a818f746cd2fea7016cd3ace4c74d29","parentKey":"8a818f746cd2fea7016cd3ace4c74d28","city":"Liverpool","country":"UK","indexInList":0}],"idDocuments":[]}


In [209]:
%%writefile NEWCREDITARRANGEMENT
{
    "amount": 10000.0,
    "availableCreditAmount": 10000.0,
    "expireDate": "2020-08-23T00:00:00+02:00",
    "exposureLimitType": "OUTSTANDING_AMOUNT",
    "holderKey": "{{CustomerID}}",
    "holderType": "CLIENT",  
    "notes": "",
    "startDate": "2019-08-23T00:00:00+02:00",
    "state": "APPROVED",
    "_URepayOptions": {
            "AutoRepayMethod": "Direct-Debit",
            "PaymentDueDay": "1",
            "ShortMonthOption": "late"
        }
}

Overwriting NEWCREDITARRANGEMENT


In [234]:
def createNewCreditArrangement():
    global CAID
    PLACEHOLDERS = {
        "CustomerID": newCustomerID
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/creditarrangements', headers=HEADERS, params=PARAMS, body="NEWCREDITARRANGEMENT", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    CAID = r.json()['id']

In [235]:
createNewCreditArrangement()

API status:
201
Now the JSON:
{"encodedKey":"8a818f796cd39234016cd3d082dd0384","id":"ZUG055","holderType":"CLIENT","holderKey":"8a818f746cd2fea7016cd3ace4c74d28","amount":10000.0,"availableCreditAmount":10000.0,"consumedCreditAmount":0.0,"state":"APPROVED","creationDate":"2019-08-27T18:02:33+02:00","lastModifiedDate":"2019-08-27T18:02:33+02:00","startDate":"2019-08-23T00:00:00+02:00","expireDate":"2020-08-23T00:00:00+02:00","approvedDate":"2019-08-27T18:02:33+02:00","exposureLimitType":"OUTSTANDING_AMOUNT","notes":"","_URepayOptions":{"ShortMonthOption":"late","PaymentDueDay":"1","AutoRepayMethod":"Direct-Debit"}}


### Create Accounts for the Customer

In [211]:
%%writefile CURACCTEMP
{
  "overdraftInterestSettings": {
        "interestRateSettings": {
            "encodedKey": "8a818f9f6cc14d6e016cc2fcdbef4923",
            "interestChargeFrequency": "ANNUALIZED",
            "interestChargeFrequencyCount": 1,
            "interestRateReviewCount": 31,
            "interestRateReviewUnit": "DAYS",
            "interestRateSource": "INDEX_INTEREST_RATE",
            "interestRateTerms": "FIXED",
            "interestSpread": 0.0
        }
    },
    "overdraftSettings": {
        "allowOverdraft": true,
        "overdraftExpiryDate": "2020-05-02T00:00:00+02:00",
        "overdraftLimit": 0.0
    },
  
  "accountType": "CURRENT_ACCOUNT",
  "name": "Current Account (with overdraft)",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "8a818f5f6cbe6621016cbf4f2db95756",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}


Writing CURACCTEMP


In [252]:
def createCurrentAccount():
    global CurrentAccID
    PLACEHOLDERS = {
        "CustomerID": newCustomerID
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/deposits', headers=HEADERS, params=PARAMS, body="CURACCTEMP", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    CurrentAccID = r.json()['id']


In [237]:
createCurrentAccount()

API status:
201
Now the JSON:
{"encodedKey":"8a818eed6cd2de01016cd3d186145cf8","creationDate":"2019-08-27T18:03:39+02:00","lastModifiedDate":"2019-08-27T18:03:39+02:00","id":"RGBG661","name":"Current Account (with overdraft)","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","accountType":"CURRENT_ACCOUNT","currencyCode":"EUR","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0.0,"overdraftExpiryDate":"2020-05-02T00:00:00+02:00"},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a818eed6cd2de01016cd3d186145cf9","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestSpread":0.0,"interestRateReviewCount":31,"interestRa

In [238]:
def addCurrentAccountToCreditLine():
    HEADERS = {'Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}'+'/linesofcredit/{0}/savings/{1}'.format(CAID,CurrentAccID) , headers=HEADERS, params=PARAMS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)

In [239]:
addCurrentAccountToCreditLine()

API status:
200
Now the JSON:
{"encodedKey":"8a818eed6cd2de01016cd3d186145cf8","id":"RGBG661","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountHolderType":"CLIENT","name":"Current Account (with overdraft)","creationDate":"2019-08-27T16:03:39+0000","lastModifiedDate":"2019-08-27T16:03:39+0000","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","lineOfCreditKey":"8a818f796cd39234016cd3d082dd0384","accountType":"CURRENT_ACCOUNT","accountState":"PENDING_APPROVAL","balance":"0","accruedInterest":"0","overdraftInterestAccrued":"0","technicalOverdraftInterestAccrued":"0","overdraftAmount":"0","technicalOverdraftAmount":"0","overdraftExpiryDate":"2020-05-02T00:00:00+0000","overdraftInterestSettings":{"interestSpread":"0","encodedKey":"8a818eed6cd2de01016cd3d186145cf9","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateSource":"INDEX_INTEREST_RATE","interestRateReviewUnit":"DAYS","interestRateReviewCount":31,"interestRateTerms":"FIXED","interestRa

In [214]:
%%writefile APPROVEACC
{
  "action": "APPROVE",
  "notes": "Approved from the API"
}

Writing APPROVEACC


In [240]:
def approveDepositAccount():
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env4}}" + "/deposits/{0}:changeState".format(CurrentAccID)
    r = POST(url, headers=HEADERS, params=PARAMS, body="APPROVEACC")
    print(r.text)

In [241]:
approveDepositAccount()

{"encodedKey":"8a818eed6cd2de01016cd3d186145cf8","creationDate":"2019-08-27T18:03:39+02:00","lastModifiedDate":"2019-08-27T18:05:35+02:00","id":"RGBG661","name":"Current Account (with overdraft)","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountState":"APPROVED","productTypeKey":"8a818f5f6cbe6621016cbf4f2db95756","accountType":"CURRENT_ACCOUNT","creditArrangementKey":"8a818f796cd39234016cd3d082dd0384","approvedDate":"2019-08-27T18:05:36+02:00","currencyCode":"EUR","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0E-10,"overdraftExpiryDate":"2020-05-02T00:00:00+02:00"},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a818eed6cd2de01016cd3d186145cf9","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIX

## Create the CC bucket accounts


In [216]:
%%writefile CCACCTEMP
{
  "loanAmount": 30000.0,
  "loanName": "{{AccountName}}",
  "accountHolderKey": "{{CustomerID}}",
  "productTypeKey": "{{ProductKey}}",
  "accountHolderType": "CLIENT",
  "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
  "interestFromArrearsAccrued": 0.0,
    "interestSettings": {
        "accrueInterestAfterMaturity": false,
        "interestApplicationMethod": "REPAYMENT_DUE_DATE",
        "interestBalanceCalculationMethod": "ONLY_PRINCIPAL",
        "interestCalculationMethod": "DECLINING_BALANCE",
        "interestChargeFrequency": "ANNUALIZED",
        "interestRateReviewCount": 31,
        "interestRateReviewUnit": "DAYS",
        "interestRateSource": "INDEX_INTEREST_RATE",
        "interestSpread": 0.0,
        "interestType": "SIMPLE_INTEREST"
    },
  "scheduleSettings": {
        "fixedDaysOfMonth": [
            1
        ],
        "gracePeriod": 0,
        "gracePeriodType": "NONE",
        "paymentPlan": [],
        "periodicPayment": 0.0,
        "principalRepaymentInterval": 1,
        "repaymentPeriodUnit": "DAYS",
        "repaymentScheduleMethod": "DYNAMIC",
        "scheduleDueDatesMethod": "FIXED_DAYS_OF_MONTH",
        "shortMonthHandlingMethod": "LAST_DAY_IN_MONTH"
    }
  
}

Writing CCACCTEMP


In [245]:
def createCreditCardBucket(cctype):
    global cc_account
    if cctype == "cash":
        productKey = "8a818f5f6cbe6621016cbf3cf8675424"
        accName = "CC - Cash"
    else:
        productKey = "8a818f5f6cbe6621016cbf6d66075e54"
        accName = "CC - Purchases"
    PLACEHOLDERS = {
        'CustomerID': newCustomerID,
        'ProductKey': productKey,
        'AccountName': accName
    }
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}/loans', headers=HEADERS, params=PARAMS, body="CCACCTEMP", bodyparts=PLACEHOLDERS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    cc_account = r.json()['id']

In [246]:
createCreditCardBucket("purchase")

API status:
201
Now the JSON:
{"encodedKey":"8a818fd26cd2df22016cd3995fea4c35","id":"LVYN893","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","creationDate":"2019-08-27T18:11:01+02:00","lastModifiedDate":"2019-08-27T18:11:01+02:00","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","loanName":"CC - Purchases","loanAmount":30000.0,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0,"interestFromArrearsAccrued":0.0,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818fd26cd2df22016cd3d84276562e","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0,"principalDue":0,"principalPaid":0,"principalBalance":0,"interestDue":0,"inte

In [247]:
def addCCBucketToCreditLine():
    HEADERS = {'Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env4}}'+'/linesofcredit/{0}/loans/{1}'.format(CAID,cc_account) , headers=HEADERS, params=PARAMS)
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)

In [248]:
addCCBucketToCreditLine()

API status:
200
Now the JSON:
{"encodedKey":"8a818fd26cd2df22016cd3995fea4c35","id":"LVYN893","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","accountHolderType":"CLIENT","creationDate":"2019-08-27T16:11:01+0000","lastModifiedDate":"2019-08-27T16:11:01+0000","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","lineOfCreditKey":"8a818f796cd39234016cd3d082dd0384","loanName":"CC - Purchases","loanAmount":"30000","periodicPayment":"0","tranches":[],"principalDue":"0","principalPaid":"0","principalBalance":"0","redrawBalance":"0","interestDue":"0","interestPaid":"0","interestFromArrearsBalance":"0","interestFromArrearsDue":"0","interestFromArrearsPaid":"0","interestBalance":"0","feesDue":"0","feesPaid":"0","feesBalance":"0","penaltyDue":"0","penaltyPaid":"0","penaltyBalance":"0","scheduleDueDatesMethod":"FIXED_DAYS_OF_MONTH","prepaymentAcceptance":"ACCEPT_PREPAYMENTS","futurePaymentsAcceptance":"NO_FUTURE_PAYMENTS","hasCustomSchedule":false,"fixedDays

In [219]:
%%writefile APPROVEACC
{
  "action": "APPROVE",
  "notes": "Approved from the API"
}

Overwriting APPROVEACC


In [250]:
def approveLoanAccount():
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env4}}" + "/loans/{0}:changeState".format(cc_account)
    r = POST(url, headers=HEADERS, params=PARAMS, body="APPROVEACC")
    print(r.text)

In [251]:
approveLoanAccount()

{"encodedKey":"8a818fd26cd2df22016cd3995fea4c35","id":"LVYN893","accountHolderType":"CLIENT","accountHolderKey":"8a818f746cd2fea7016cd3ace4c74d28","creationDate":"2019-08-27T18:11:01+02:00","approvedDate":"2019-08-27T18:12:29+02:00","lastModifiedDate":"2019-08-27T18:12:28+02:00","accountState":"APPROVED","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","creditArrangementKey":"8a818f796cd39234016cd3d082dd0384","loanName":"CC - Purchases","loanAmount":30000.0000000000,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0E-10,"interestFromArrearsAccrued":0E-10,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818fd26cd2df22016cd3d84276562e","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0

<a id="section1"></a>
[back](#sectionx)
## Add New Card Transaction

In [202]:
%%writefile NEWCARDTRANS
{
    "Amount": "10",
    "DebitCardAvailable": "1000",
    "DebitCardOnly": "F",
    "Merchant": "Starbucks",
    "POSCreditTypeChoice": "0",
    "PurchaseCategory": "Food+Drink",
    "transactionID": "{{TranID}}"
}

Overwriting NEWCARDTRANS


In [204]:
PLACEHOLDERS = {
    "TranID": getUUID()
}
HEADERS = {}
PARAMS = {}
r = POST('https://mpo-sandbox.mambucloud.com/api/1/json/public/18614/28388948acbf477da3d99dc3999e264ab745965a', 
         headers=HEADERS, params=PARAMS, body="NEWCARDTRANS", bodyparts=PLACEHOLDERS)
PRINT(r)

API status: 200
JSON Response:
{
    "ops": {
        "obj": "task",
        "obj_id": "5d654f0a09a8af0fc8002b63",
        "proc": "ok",
        "ref": null
    },
    "request_proc": "ok"
}


## Miscellaneous


In [59]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON', 'limit': 10}
r = GET('{{env4}}/creditarrangements', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
[
    {
        "_URepayOptions": {
            "AutoRepayMethod": "Direct-Debit",
            "PaymentDueDay": "1",
            "ShortMonthOption": "late"
        },
        "amount": 10000.0,
        "approvedDate": "2019-08-27T14:37:27+02:00",
        "availableCreditAmount": 10000.0,
        "consumedCreditAmount": 0.0,
        "creationDate": "2019-08-27T14:37:27+02:00",
        "encodedKey": "8a818f156ccf5fb1016cd2ef00482b47",
        "expireDate": "2020-09-10T00:00:00+02:00",
        "exposureLimitType": "APPROVED_AMOUNT",
        "holderKey": "8a818faf6cd2dc01016cd30cd28702e4",
        "holderType": "CLIENT",
        "id": "HGO606",
        "lastModifiedDate": "2019-08-27T14:37:27+02:00",
        "notes": "",
        "startDate": "2019-08-27T00:00:00+02:00",
        "state": "APPROVED"
    },
    {
        "amount": 10000.0,
        "approvedDate": "2019-08-23T18:00:29+02:00",
        "availableCreditAmount": 10000.0,
        "consumedCreditAmount

In [94]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET('{{env4}}/deposits/HRIP331', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "accountHolderKey": "8a818f5f6cbe6621016cbf30b8095261",
    "accountHolderType": "CLIENT",
    "accountState": "ACTIVE",
    "accountType": "CURRENT_ACCOUNT",
    "accruedAmounts": {
        "interestAccrued": 0.0,
        "overdraftInterestAccrued": 0.0,
        "technicalOverdraftInterestAccrued": 0.0
    },
    "activationDate": "2019-08-24T12:24:14+02:00",
    "approvedDate": "2019-08-24T12:22:23+02:00",
    "assignedBranchKey": "8a818f5f6cbe6621016cbf217c9e5060",
    "balances": {
        "availableBalance": 700.0,
        "feesDue": 0.0,
        "holdBalance": 0.0,
        "lockedBalance": 0.0,
        "overdraftAmount": 0.0,
        "overdraftInterestDue": 0.0,
        "technicalOverdraftAmount": 0.0,
        "technicalOverdraftInterestDue": 0.0,
        "totalBalance": 700.0
    },
    "creationDate": "2019-08-24T12:17:43+02:00",
    "creditArrangementKey": "8a818f5f6cbe6621016cbf352ec55343",
    "currencyCode": "EUR",
    "encodedKey": "8a8

In [178]:

HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET('{{env4}}/loans/CVFJ335', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "accountArrearsSettings": {
        "dateCalculationMethod": "ACCOUNT_FIRST_WENT_TO_ARREARS",
        "encodedKey": "8a818ece6cd2e07e016cd39ba9ad4f6a",
        "nonWorkingDaysMethod": "EXCLUDED",
        "toleranceCalculationMethod": "ARREARS_TOLERANCE_PERIOD",
        "tolerancePeriod": 0
    },
    "accountHolderKey": "8a818ece6cd2e07e016cd33b94f40832",
    "accountHolderType": "CLIENT",
    "accountState": "PENDING_APPROVAL",
    "accruedInterest": 0.0,
    "accruedPenalty": 0.0,
    "allowOffset": false,
    "arrearsTolerancePeriod": 0,
    "assets": [],
    "assignedBranchKey": "8a818f156ccf5fb1016cd2e8e4532b09",
    "balances": {
        "feesBalance": 0.0,
        "feesDue": 0.0,
        "feesPaid": 0.0,
        "holdBalance": 0.0,
        "interestBalance": 0.0,
        "interestDue": 0.0,
        "interestFromArrearsBalance": 0.0,
        "interestFromArrearsDue": 0.0,
        "interestFromArrearsPaid": 0.0,
        "interestPaid": 0.0,
   

<a id="section3"></a>
[back](#sectionx)
## Get customer Accounts

In [297]:
%%writefile LOANSEARCH
{
  "filterConstraints":[
      {
        "filterSelection":"ACCOUNT_HOLDER_KEY",
        "filterElement":"EQUALS",
        "value":"{{CustomerID}}"
      },
      {
        "filterSelection":"PRODUCT_KEY",
        "filterElement":"EQUALS",
        "value":"{{ProductKey}}"
      }
   ]
}

Overwriting LOANSEARCH


In [306]:
def getCustomerAccount(accTy, productKey):
    if accTy == "loan":
        apiStr = '{{env4}}/loans/search'
    else:
        apiStr = '{{env4}}/savings/search'
    PLACEHOLDERS = {
            "CustomerID": newCustomerID,
            "ProductKey": productKey
        }
    HEADERS = {'Content-Type': 'application/json'}
    # Parameters below set up pagination and only returns max 20
    PARAMS = {}
    r = POST(apiStr, headers=HEADERS, params=PARAMS, body="LOANSEARCH", bodyparts=PLACEHOLDERS)
    resList = r.json()
    for resItem in resList:
        print(resItem["id"])
        print(resItem["accountHolderKey"])
    return resItem["id"]

In [309]:
# CC Purchases Account
print("CC Purchases Account")
getCustomerAccount("loan","8a818f5f6cbe6621016cbf6d66075e54")

print("CC Cash Account")
getCustomerAccount("loan", "8a818f5f6cbe6621016cbf3cf8675424")

print("Current Account (with overdraft)")
getCustomerAccount("deposit", "8a818f5f6cbe6621016cbf4f2db95756")


CC Purchases Account
GNNW982
8a818f766cd3015d016cd3d8ac605e28
CC Cash Account
AUIN516
8a818f766cd3015d016cd3d8ac605e28
Current Account (with overdraft)
YMLH036
8a818f766cd3015d016cd3d8ac605e28


'YMLH036'

In [279]:
print(newCustomerID)

8a818f766cd3015d016cd3d8ac605e28


<a id="section2"></a>
[back](#sectionx)
## Create ALL in 1 go

In [265]:
def createALL(fName, lName):
    createNewCustomer(fName, lName)
    createNewCreditArrangement()
    createCurrentAccount()
    addCurrentAccountToCreditLine()
    approveDepositAccount()
    createCreditCardBucket("cash")
    addCCBucketToCreditLine()
    approveLoanAccount()
    createCreditCardBucket("purchase")
    addCCBucketToCreditLine()
    approveLoanAccount()

In [268]:
createALL("Gerraldine", "Edwards")
print("**********Finished ***********************")
print("")
print(custShortID)

API status:
201
Now the JSON:
{"encodedKey":"8a818f766cd3015d016cd3d8ac605e28","id":"078508009","state":"INACTIVE","creationDate":"2019-08-27T18:28:24+02:00","lastModifiedDate":"2019-08-27T18:28:24+02:00","approvedDate":"2019-08-27T18:28:24+02:00","firstName":"Gerraldine","lastName":"Edwards","preferredLanguage":"ENGLISH","gender":"MALE","notes":"Some Notes on this person","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","clientRoleKey":"8a818e74677a2e9201677ec2b4c336aa","loanCycle":0,"groupLoanCycle":0,"addresses":[{"encodedKey":"8a818f766cd3015d016cd3e8303c6068","parentKey":"8a818f766cd3015d016cd3d8ac605e28","city":"Liverpool","country":"UK","indexInList":0}],"idDocuments":[]}
API status:
201
Now the JSON:
{"encodedKey":"8a818eea6cd2dc28016cd3c717010176","id":"AWJ898","holderType":"CLIENT","holderKey":"8a818f766cd3015d016cd3d8ac605e28","amount":10000.0,"availableCreditAmount":10000.0,"consumedCreditAmount":0.0,"state":"APPROVED","creationDate":"2019-08-27T18:28:27+02:00","lastM

API status:
200
Now the JSON:
{"encodedKey":"8a818ece6cd2e07e016cd3e844675861","id":"AUIN516","accountHolderKey":"8a818f766cd3015d016cd3d8ac605e28","accountHolderType":"CLIENT","creationDate":"2019-08-27T16:28:30+0000","lastModifiedDate":"2019-08-27T16:28:30+0000","accountState":"PENDING_APPROVAL","productTypeKey":"8a818f5f6cbe6621016cbf3cf8675424","lineOfCreditKey":"8a818eea6cd2dc28016cd3c717010176","loanName":"CC - Cash","loanAmount":"30000","periodicPayment":"0","tranches":[],"principalDue":"0","principalPaid":"0","principalBalance":"0","redrawBalance":"0","interestDue":"0","interestPaid":"0","interestFromArrearsBalance":"0","interestFromArrearsDue":"0","interestFromArrearsPaid":"0","interestBalance":"0","feesDue":"0","feesPaid":"0","feesBalance":"0","penaltyDue":"0","penaltyPaid":"0","penaltyBalance":"0","scheduleDueDatesMethod":"FIXED_DAYS_OF_MONTH","prepaymentAcceptance":"ACCEPT_PREPAYMENTS","futurePaymentsAcceptance":"NO_FUTURE_PAYMENTS","hasCustomSchedule":false,"fixedDaysOfMon

{"encodedKey":"8a818f136cd2fe8e016cd3e63a495689","id":"GNNW982","accountHolderType":"CLIENT","accountHolderKey":"8a818f766cd3015d016cd3d8ac605e28","creationDate":"2019-08-27T18:28:34+02:00","approvedDate":"2019-08-27T18:28:36+02:00","lastModifiedDate":"2019-08-27T18:28:36+02:00","accountState":"APPROVED","productTypeKey":"8a818f5f6cbe6621016cbf6d66075e54","creditArrangementKey":"8a818eea6cd2dc28016cd3c717010176","loanName":"CC - Purchases","loanAmount":30000.0000000000,"paymentMethod":"HORIZONTAL","assignedBranchKey":"8a818f156ccf5fb1016cd2e8e4532b09","accruedInterest":0E-10,"interestFromArrearsAccrued":0E-10,"allowOffset":false,"arrearsTolerancePeriod":0,"accountArrearsSettings":{"encodedKey":"8a818f136cd2fe8e016cd3e854f656aa","toleranceCalculationMethod":"ARREARS_TOLERANCE_PERIOD","dateCalculationMethod":"ACCOUNT_FIRST_WENT_TO_ARREARS","nonWorkingDaysMethod":"EXCLUDED","tolerancePeriod":0},"latePaymentsRecalculationMethod":"OVERDUE_INSTALLMENTS_INCREASE","balances":{"redrawBalance":0

In [267]:
print(custShortID)

892870345
